In [1]:
#overarching parent class - every child class is located on a virtual private cloud
class Vpc:

    def __init__(self, sg_id, name):
        self.sg_id = sg_id
        self.name = name
        self.elb_instances = []
        self.ec2_instances = []
        self.rds_instances = []
        
#Elastic load balancer, child class of 'Vpc'
class Elb:
    
    def __init__(self, sg_id, name, vpc_id):
        self.sg_id = sg_id
        self.name = name
        self.vpc_id = vpc_id
        self.security_group_instances = []

#Elastic Compute Cloud, child class of 'Vpc'.. parent class of 'Security_Group' and 'Ebs'    
class Ec2:
    
    def __init__(self, sg_id, name, vpc_id):
        self.sg_id = sg_id
        self.name = name
        self.vpc_id = vpc_id
        self.security_group_instances = []
        self.ebs_instances = []

#Security group, child class of 'Ec2'
class Security_Group:
    
    def __init__(self, sg_id, name, vpc_id):
        self.sg_id = sg_id
        self.name = name
        self.vpc_id = vpc_id

#Elastic Block Store, child class of 'Ec2'
class Ebs:
    
    def __init__(self, sg_id, name, vpc_id, ec2_id):
        self.sg_id = sg_id
        self.name = name
        self.vpc_id = vpc_id
        self.ec2_id = ec2_id

#Relational Database service, child class of 'Vpc'
class Rbs:
    def __init__(self, sg_id, name, vpc_id):
        self.sg_id = sg_id
        #name is actually a security id
        self.name = name
        self.vpc_id = vpc_id
        self.rbs_instances = []
        self.vpc_security_groups= []

Import data from VPC's json file and build a VPC object called 'vpc_instance'

In [2]:

import json
f = open("vpc.json", "r")
s = f.read()
f.close()
vpcs = json.loads(s)

In [3]:
global_vpc_dict = {}
for vpc in vpcs['Vpcs']:
    vpc_id = vpc["VpcId"]
    vpc_name = ""
    for tag in vpc['Tags']:
        if (tag["Key"] == "Name"):
            vpc_name = tag["Value"]
            break
    vpc_instance = Vpc(vpc_id, vpc_name)
    global_vpc_dict[vpc_id] = vpc_instance

Import data from elb's json file and build a elb object called 'elb_instance'
Then add 'elb_instance' to the object 'vpc_instance'

In [4]:
import json
f = open("elb.json", "r")
s = f.read()
f.close()
elbs = json.loads(s)

In [5]:
for elb in elbs['LoadBalancerDescriptions']:
    elb_name = elb['LoadBalancerName']
    elb_id = elb_name
    vpc_id = elb['VPCId']
    elb_instance = Elb(elb_id, elb_name, vpc_id)
    
    security_group_id = elb['SecurityGroups'][0]
    security_group_name = security_group_id
    security_group_instance = Security_Group(security_group_id, security_group_name, vpc_id)
    elb_instance.security_group_instances.append(security_group_instance)
    vpc_instance = global_vpc_dict[vpc_id]
    vpc_instance.elb_instances.append(elb_instance)

Import data from elbv2's json file and build a elb object called 'elb_instance'
Then add 'elb_instance' to the object 'vpc_instance'
*elbv2 differs in that it is AWS' newer version of elb, Immport utilizes both versions

In [6]:
import json
f = open("elbv2.json", "r")
s = f.read()
f.close()
elbs = json.loads(s)

In [7]:
for elb in elbs['LoadBalancers']:
    elb_name = elb['LoadBalancerName']
    elb_id = elb_name
    vpc_id = elb['VpcId']
    elb_instance = Elb(elb_id, elb_name, vpc_id)
    
    security_group_id = elb['SecurityGroups'][0]
    security_group_name = security_group_id
    security_group_instance = Security_Group(security_group_id, security_group_name, vpc_id)
    elb_instance.security_group_instances.append(security_group_instance)
    vpc_instance = global_vpc_dict[vpc_id]
    vpc_instance.elb_instances.append(elb_instance)

Import data from rds's json file and build a rds object called 'rds_instance'
Then add 'rds_instance' to the object 'vpc_instance'

In [8]:
# import json
with open("rds.json", "r") as rfile:
    rds_info = rfile.read()

    rds = json.loads(rds_info)



In [9]:
#construct object for Rds

global_rds_dict = {}
for instance in rds['DBInstances']:
    #   get vpc_id
    DBSubGr = instance['DBSubnetGroup']
    vpc_id = DBSubGr['VpcId']
    
    #get dbinstanceidneitifier
    db_id = instance['DBInstanceIdentifier']

    #get vpc security group
    sg_id = instance['DbiResourceId']

    
  
    Rds_instance = Rbs(sg_id, db_id, vpc_id)
    
    global_rds_dict[db_id] = Rds_instance 
    global_vpc_dict[vpc_id].rds_instances.append(Rds_instance)


Import data from ec2's json file and build a ec2 object called 'ec2_instance'
Then add 'ec2_instance' to the object 'vpc_instance'

In [10]:
f = open("ec2.json", "r")
s = f.read()
f.close()
ec2s = json.loads(s)

In [11]:
#create a dictionary to quickly access the many unique ec2 objects: key is 'Key' (in json file), value is 'Value' (in json file)
global_ec2_dict = {}

#iterate through the json file for relevant information
for reservation in ec2s['Reservations']:
    for instance in reservation['Instances']:
        ec2_id = instance["InstanceId"]
        ec2_name = ""
        for tag in instance['Tags']:
            if (tag["Key"] == "Name"):
                ec2_name = tag["Value"]
                break 
        if (instance.get("VpcId")):
            vpc_id = instance["VpcId"]
    
            ec2_instance = Ec2(ec2_id, ec2_name, vpc_id)
            global_ec2_dict[ec2_id] = ec2_instance
            global_vpc_dict[vpc_id].ec2_instances.append(ec2_instance)
           
            security_group_id = instance['SecurityGroups'][0]['GroupId']
            security_group_name = instance['SecurityGroups'][0]['GroupName']
            security_group_instance = Security_Group(security_group_id, security_group_name, vpc_id)
            ec2_instance.security_group_instances.append(security_group_instance)
    
            for block_device in instance['BlockDeviceMappings']:
                ebs_id = block_device["Ebs"]["VolumeId"]
                ebs_name = block_device["DeviceName"]
                ebs_instance = Ebs(ebs_id, ebs_name, vpc_id, ec2_id)
                ec2_instance.ebs_instances.append(ebs_instance)
        else:
            print("*** Instance " + instance.get("InstanceId") + " does not belong to a VPC. ***")

In [12]:
import operator

#
# Process each VPC
#
vpc_list = []
for vpc_instance in (sorted(global_vpc_dict.values(), key=operator.attrgetter('name'))):

    vpc_dict = { "id" : vpc_instance.sg_id, "name" : vpc_instance.name }

    #
    # Process each ELB
    #
    elb_list = []
    for elb_instance in (sorted(vpc_instance.elb_instances, key=lambda x: x.name)):
        #
        # Process each security group
        #
        security_group_list = []
        for security_group_instance in (sorted(elb_instance.security_group_instances, key=lambda x: x.name)):
            security_group_id = security_group_instance.sg_id
            security_group_name = security_group_instance.name
            security_group_dict = { "id" : security_group_id, "name" : security_group_name }
            security_group_list.append(security_group_dict)
            
        elb_dict = { "id" : elb_instance.sg_id, "name" : elb_instance.name, "security_group_list" : security_group_list }
        elb_list.append(elb_dict)
            
    #
    # Process each EC2
    #
    ec2_list = []
    for ec2_instance in (sorted(vpc_instance.ec2_instances, key=lambda x: x.name)):
        #
        # Process each security group
        #
        security_group_list = []
        for security_group_instance in (sorted(ec2_instance.security_group_instances, key=lambda x: x.name)):
            security_group_id = security_group_instance.sg_id
            security_group_name = security_group_instance.name
            security_group_dict = { "id" : security_group_id, "name" : security_group_name }
            security_group_list.append(security_group_dict)

        #
        # Process each EBS
        #
        ebs_list = []
        for ebs_instance in (sorted(ec2_instance.ebs_instances, key=lambda x: x.name)):
            ebs_dict = { "id" : ebs_instance.sg_id, "name" : ebs_instance.name }
            ebs_list.append(ebs_dict)

        ec2_dict = { "id" : ec2_instance.sg_id, "name" : ec2_instance.name, "security_group_list" : security_group_list, "ebs_list" : ebs_list }
        ec2_list.append(ec2_dict)
    
    vpc_dict["elb_list"] = elb_list
    vpc_dict["ec2_list"] = ec2_list
    vpc_list.append(vpc_dict)
    
aws_dict = {}
aws_dict["vpc_list"] = vpc_list

print(aws_dict)

{'vpc_list': [{'id': 'vpc-d2e999b6', 'name': 'ImmportDev', 'ec2_list': [{'id': 'i-054feb0b45296469c', 'name': 'ImmportDevApi', 'ebs_list': [{'id': 'vol-011e1dff3a2937922', 'name': '/dev/sda1'}, {'id': 'vol-08faf47cea9f1794d', 'name': '/dev/sdf'}], 'security_group_list': [{'id': 'sg-40d4ec3e', 'name': 'ImmportDevApi'}]}, {'id': 'i-0b2d47d2bcca497e4', 'name': 'ImmportDevApp', 'ebs_list': [{'id': 'vol-052e52d04e7f28028', 'name': '/dev/sda1'}, {'id': 'vol-01adade34662454d8', 'name': '/dev/sdd'}, {'id': 'vol-0fb9a751f8b5788bd', 'name': '/dev/sdf'}], 'security_group_list': [{'id': 'sg-899915f8', 'name': 'ImmportDevApp'}]}, {'id': 'i-030007dae4dc87ee9', 'name': 'ImmportDevAppScan', 'ebs_list': [{'id': 'vol-04720aaaefc33cf2d', 'name': '/dev/sda1'}], 'security_group_list': [{'id': 'sg-1655c566', 'name': 'ImmportDevAppScan'}]}, {'id': 'i-0121d443b7c486bbd', 'name': 'ImmportDevAspera', 'ebs_list': [{'id': 'vol-07960ac7cb97de7b9', 'name': '/dev/sda1'}, {'id': 'vol-055a4d4b9e8b4b9f8', 'name': '/dev

In [13]:
aws = json.dumps(aws_dict, indent=4)
print(aws)

{
    "vpc_list": [
        {
            "id": "vpc-d2e999b6",
            "name": "ImmportDev",
            "ec2_list": [
                {
                    "id": "i-054feb0b45296469c",
                    "name": "ImmportDevApi",
                    "ebs_list": [
                        {
                            "id": "vol-011e1dff3a2937922",
                            "name": "/dev/sda1"
                        },
                        {
                            "id": "vol-08faf47cea9f1794d",
                            "name": "/dev/sdf"
                        }
                    ],
                    "security_group_list": [
                        {
                            "id": "sg-40d4ec3e",
                            "name": "ImmportDevApi"
                        }
                    ]
                },
                {
                    "id": "i-0b2d47d2bcca497e4",
                    "name": "ImmportDevApp",
                    "ebs_list": [
    

In [14]:
with open("aws.json", "w") as f:
    f.write(aws)
    f.close()

In [15]:
from treelib import Tree, Node

tree = Tree()
root_node = tree.create_node("VPCs","root_node")
        
#
# Process each VPC
#
for vpc_instance in (sorted(global_vpc_dict.values(), key=operator.attrgetter('name'))):

    vpc_node_label = "VPC: " + vpc_instance.name + " (" + vpc_instance.sg_id + ")"
    vpc_node = Node(vpc_node_label, vpc_instance.sg_id)
    tree.add_node(vpc_node, "root_node")

    #
    # Process each ELB
    #
    for elb_instance in (sorted(vpc_instance.elb_instances, key=lambda x: x.name)):
        elb_node_label = "ELB: " + elb_instance.name
        elb_node = Node(elb_node_label, elb_instance.sg_id)
        tree.add_node(elb_node, vpc_instance.sg_id)
        #
        # Process each security group
        #
        for security_group_instance in (sorted(elb_instance.security_group_instances, key=lambda x: x.name)):
            security_group_id = security_group_instance.sg_id
            security_group_name = security_group_instance.name
            security_group_node_label = "Security Group: " + security_group_name + " (" + security_group_id + ")"
            security_group_node_id = elb_instance.sg_id + ":" + security_group_id
            security_group_node = Node(security_group_node_label, security_group_node_id)
            tree.add_node(security_group_node, elb_instance.sg_id)
    
    #
    # Process each EC2
    #
    for ec2_instance in (sorted(vpc_instance.ec2_instances, key=lambda x: x.name)):
        ec2_node_label = "EC2: " + ec2_instance.name + " (" + ec2_instance.sg_id + ")"
        ec2_node = Node(ec2_node_label, ec2_instance.sg_id)
        tree.add_node(ec2_node, vpc_instance.sg_id)
        #
        # Process each security group
        #
        for security_group_instance in (sorted(ec2_instance.security_group_instances, key=lambda x: x.name)):
            security_group_id = security_group_instance.sg_id
            security_group_name = security_group_instance.name
            security_group_node_label = "Security Group: " + security_group_name + " (" + security_group_id + ")"
            security_group_node_id = ec2_instance.sg_id + ":" + security_group_id
            security_group_node = Node(security_group_node_label, security_group_node_id)
            tree.add_node(security_group_node, ec2_instance.sg_id)
        #
        # Process each EBS
        #
        for ebs_instance in (sorted(ec2_instance.ebs_instances, key=lambda x: x.name)):
            ebs_id = ebs_instance.sg_id
            ebs_name = ebs_instance.name
            ebs_node_label = "EBS: " + ebs_name + " (" + ebs_id + ")"
            ebs_node_id = ec2_instance.sg_id + ":" + ebs_id
            ebs_node = Node(ebs_node_label, ebs_node_id)
            tree.add_node(ebs_node, ec2_instance.sg_id)
            
    for rds_instance in (sorted(vpc_instance.rds_instances, key=lambda x: x.name)):
        rds_node_label = "RDS: " + rds_instance.name + " (" + rds_instance.sg_id + ")"
        rds_node = Node(rds_node_label, rds_instance.sg_id)
        tree.add_node(rds_node, vpc_instance.sg_id)


In [16]:
tree.show("root_node",0,1)

VPCs
├── VPC: ImmportDev (vpc-d2e999b6)
│   ├── EC2: ImmportDevApi (i-054feb0b45296469c)
│   │   ├── EBS: /dev/sda1 (vol-011e1dff3a2937922)
│   │   ├── EBS: /dev/sdf (vol-08faf47cea9f1794d)
│   │   └── Security Group: ImmportDevApi (sg-40d4ec3e)
│   ├── EC2: ImmportDevApp (i-0b2d47d2bcca497e4)
│   │   ├── EBS: /dev/sda1 (vol-052e52d04e7f28028)
│   │   ├── EBS: /dev/sdd (vol-01adade34662454d8)
│   │   ├── EBS: /dev/sdf (vol-0fb9a751f8b5788bd)
│   │   └── Security Group: ImmportDevApp (sg-899915f8)
│   ├── EC2: ImmportDevAppScan (i-030007dae4dc87ee9)
│   │   ├── EBS: /dev/sda1 (vol-04720aaaefc33cf2d)
│   │   └── Security Group: ImmportDevAppScan (sg-1655c566)
│   ├── EC2: ImmportDevAspera (i-0121d443b7c486bbd)
│   │   ├── EBS: /dev/sda1 (vol-07960ac7cb97de7b9)
│   │   ├── EBS: /dev/sdb (vol-055a4d4b9e8b4b9f8)
│   │   ├── EBS: /dev/sdf (vol-05ed9a45fe4c813b4)
│   │   └── Security Group: ImmportDevAspera (sg-3f71d34f)
│   ├── EC2: ImmportDevAuth (i-a2161a11)
│   │   ├── EBS: /dev/sda1 (vol